In [14]:
import xlrd
import xlwt
from xlwt import Workbook
from selenium import webdriver
import glob
import os
from selenium.webdriver.support.ui import Select
import time
import requests as req
from bs4 import BeautifulSoup
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows


#DataBase excel storage
path = "C:/Users/Raja/Mydata.xlsx"
wb_obj = openpyxl.Workbook() 
sheet_obj = wb_obj.active


#title for excel
sheet_obj.cell(row= 1 , column = 1).value="LIST OF COMPANIES"
sheet_obj.cell(row= 1 , column = 2).value="Company URL"
sheet_obj.cell(row= 1 , column = 3).value="Company DIN Number"
sheet_obj.cell(row= 1 , column = 4).value="Director Name"


#Selenium Webdriver
web = webdriver.Chrome()

#Cibil navigate page(credit 1st)
web.get('https://suit.cibil.com/')
select = Select(web.find_element_by_xpath('//*[@id="quarterIdGrantors"]'))
select.select_by_visible_text('31-03-20')
web.find_element_by_xpath('//*[@id="loadSuitFiledDataSearchAction"]/div[1]/div[2]/div[3]/img').click()
#download
web.find_element_by_xpath('//*[@id="downloadReport"]/div').click()

#Directors of company
def addDirectors(company,link,x):
    if link==None:
        c4=sheet_obj.cell(row= x-1 , column = 4)
        reply="Director NOT AVAILABLE"
        c4.value=reply
    else:    
        link=link.replace("company","company-directors")
        r=req.get(link)
        soup=BeautifulSoup(r.content)
        attr={"class":"table table-striped"}
        names=[]
        #wb=Workbook()
        #sheet=wb.add_sheet('Directors')
        for i in soup.find_all("table",attr):
            for j in i.find_all("tr"):
                for k in j.find_all("td"):
                    for l in k.find_all("h5"):
                        for m in k.find_all("a"):
                            if m.text.isupper():
                                names.append(m.text)
        
        
        for i,e in enumerate(names):
            c4=sheet_obj.cell(row= x-1 , column = 4)
            if e==None or e=='':
                c4.value="No Director"
            else:
                if c4.value=='' or c4.value==None:
                    c4.value=e
                else:
                    c4.value=c4.value+', '+e
        print(names)


#Gets Company URL
def companySearch(s):
    if s!="":
        company=s.replace(" ","-")
        company=company.lower()
        
        url="https://www.zaubacorp.com/companysearchresults/"+company
        attr={"class":"container basic-cont"}
        r=req.get(url)
        soup=BeautifulSoup(r.content)


        l=soup.find_all("div",attr)
        child=list(l[0].children)
        text=str(child[2])
        if text[4:len(text)-4].startswith("We are unable to find"):
            return None
            
        else:
            #print("Company found")
            l=soup.find_all("h5")
            
            id=str(l[0])[4:len(l[0])-6]
            #print(id)
            
            url="https://www.zaubacorp.com/company/"+company.upper()+"/"+id
            #print(s.upper())
            #print(url)
            
            return url

#Returns DIN Number
def dinSearch(s):
    if s!="":
        company=s.replace(" ","-")
        company=company.lower()
        
        url="https://www.zaubacorp.com/companysearchresults/"+company
        attr={"class":"container basic-cont"}
        r=req.get(url)
        soup=BeautifulSoup(r.content)


        l=soup.find_all("div",attr)
        child=list(l[0].children)
        text=str(child[2])
        if text[4:len(text)-4].startswith("We are unable to find"):
            return None
            
        else:
            #print("Company found")
            l=soup.find_all("h5")
            
            id=str(l[0])[4:len(l[0])-6]
            #print(id)
            return id
        

#Gets the location of the last modified folder       
list_of_files = glob.glob('C:/Users/Raja/Downloads/*')
latest_file = max(list_of_files, key=os.path.getctime)
print (latest_file)

loc = latest_file.replace("\\","/")
wb=xlrd.open_workbook(loc)

sheet=wb.sheet_by_index(0)


for i in range(3,sheet.nrows-1):
    c1=sheet_obj.cell(row= i-1 , column = 1)
    c1.value= sheet.cell_value(i,1)
    
    rtn=companySearch(sheet.cell_value(i,1))
    c2=sheet_obj.cell(row= i-1 , column = 2)
    c2.value=rtn
    
    rtn=dinSearch(sheet.cell_value(i,1))
    c3=sheet_obj.cell(row= i-1 , column = 3)
    c3.value=rtn
    
    l=companySearch(sheet.cell_value(i,1))
    addDirectors(sheet.cell_value(i,1),l,i)
    #print(c2.value)
    
    

wb_obj.save(path)


C:/Users/Raja/Downloads\cgSummaryReport_1601042730701.xls
[' ANOOP KUMAR PRABHAKAR', ' IAN DOWNING']
[' LALITHA RAMESWARAN', ' BISWAJIT RAY', ' ARUMUGAM MANIMEKHALAI', ' CHANDRAMOHAN DEVULAPALLE REDDY']
[' GOVIND BALLABH PANDE', ' DAKSHITA DAS', ' ATANU KUMAR DAS', ' AMARENDRA PRATAP SINGH', ' RAMASAMY VENKATARAMAN']
[' BIRANCHI NARAYAN NAYAK', ' SURESH KUMAR JAIN', ' BIKASH KANTI ROY', ' EMANDI SANKARA RAO', ' ARVIND KUMAR JAIN', ' MADHUSHREE NANDA AGARWAL']
[' ARVIND SAHAY', ' ANAND MADHUKAR', ' MADAN MOHANLAL VERMA']
[' VIRENDRA KUMAR DHINGRA', ' SHASHANK SAKSENA', ' SUNANDA LAHIRI']
[' JAGDISH CAPOOR', ' SIDDHARTHA MOHANTY', ' AMEET NAVINCHANDRA PATEL', ' KASHI PRASAD KHANDELWAL', ' DHARMENDRA BHANDARI', ' VIPAN KUMAR KUKREJA', ' VIPIN ANAND', ' MANGALAM RAMASUBRAMANIAN KUMAR', ' POTTIMUTYALA KOTESWARA RAO', ' SAVITA PARAM BIR SINGH']
[' RAJEEV SHARMA', ' RAM CHANDRA MISHRA', ' GOURI CHAUDHURY', ' RAVINDER SINGH DHILLON']
[' BHARATI RAO', ' VINAY SUDHAKARRAO HEDAOO', ' ARAVINDKUMAR